In [16]:
import pandas as pd
from pandas import  DataFrame
from sklearn.model_selection import train_test_split

In [17]:
df =pd.read_csv('./data/train.csv', encoding = "ISO-8859-1")

In [23]:
labels= df.iloc[:,0]
trainX= df.iloc[:,1]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(trainX, labels, train_size = 0.8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
# import and instantiate TfidfVectorizer (with the default parameters)
# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams
vect.set_params(ngram_range=(1, 2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# only keep terms that appear in at least 2 documents
vect.set_params(min_df=2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000001F05D334668>>,
        use_idf=True, vocabulary=None)

In [29]:
# learn the 'vocabulary' of the training data
vect.fit(X_train)

# examine the fitted vocabulary
#vect.get_feature_names()[:50]

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000001F05D334668>>,
        use_idf=True, vocabulary=None)

In [30]:
train_vectors = vect.transform(X_train)
train_vectors

<24000x34825 sparse matrix of type '<class 'numpy.float64'>'
	with 568281 stored elements in Compressed Sparse Row format>

In [31]:
test_vectors = vect.transform(X_test)
test_vectors

<6000x34825 sparse matrix of type '<class 'numpy.float64'>'
	with 131623 stored elements in Compressed Sparse Row format>

In [32]:
pd.DataFrame(train_vectors.toarray(), columns=vect.get_feature_names()).head(4)

!  ! !  ! #  ! &  ! 'll  ! 'm  ! 're  ! ,  ! .  ! .*t        ...          \
0  0.0  0.0  0.0  0.0    0.0   0.0    0.0  0.0  0.0    0.0        ...           
1  0.0  0.0  0.0  0.0    0.0   0.0    0.0  0.0  0.0    0.0        ...           
2  0.0  0.0  0.0  0.0    0.0   0.0    0.0  0.0  0.0    0.0        ...           
3  0.0  0.0  0.0  0.0    0.0   0.0    0.0  0.0  0.0    0.0        ...           

   zopnow  zopnow ,  zoppies    {    |  | cashback  | learning    }  ~was  \
0     0.0       0.0      0.0  0.0  0.0         0.0         0.0  0.0   0.0   
1     0.0       0.0      0.0  0.0  0.0         0.0         0.0  0.0   0.0   
2     0.0       0.0      0.0  0.0  0.0         0.0         0.0  0.0   0.0   
3     0.0       0.0      0.0  0.0  0.0         0.0         0.0  0.0   0.0   

   ~was successfully  
0                0.0  
1                0.0  
2                0.0  
3                0.0  

[4 rows x 34825 columns]

In [33]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = GaussianNB()

In [34]:
nb.fit(train_vectors.toarray(), y_train)

GaussianNB(priors=None)

In [35]:
from sklearn.metrics.classification import classification_report, accuracy_score
from sklearn.cross_validation import cross_val_predict

# generate your cross-validation prediction with 10 fold Stratified sampling
y_pred = cross_val_predict(nb, test_vectors.toarray(), y_test, cv=10)

print(classification_report(y_test, y_pred))
print("ACCURACY::", accuracy_score(y_pred, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


             precision    recall  f1-score   support

        ham       0.99      0.95      0.97      2031
       info       0.99      1.00      1.00      2648
       spam       0.94      0.99      0.96      1321

avg / total       0.98      0.98      0.98      6000

ACCURACY:: 0.979333333333
